In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Specifying incognito mode as you launch your browser[OPTIONAL]
option = webdriver.ChromeOptions()
option.add_argument("--no-sandbox")

browser = webdriver.Chrome("/home/pahul/Documents/chromedriver")

companies_titles = ['groove',"slack"]
source_site_title = 'https://www.trustradius.com/'


data_final=pd.DataFrame()
for company_title in companies_titles:
    browser.get('{}products/{}/reviews/pros-and-cons'.format(source_site_title, company_title))
    
    #pros cons data
    pros_cons_data=[]
    data_list= browser.find_elements_by_class_name("serp-details")
    
    for review in data_list:
        try:
            title=review.find_element_by_class_name("review-title").text

            name=review.find_element_by_class_name("name").text

            company=review.find_element_by_class_name("company").text

            score=review.find_element_by_class_name("trust-score__score").text

            pros_list=review.find_element_by_class_name("pros").find_elements_by_class_name("ugc")
            pros_data=[x.text for x in pros_list]

            cons_list=review.find_element_by_class_name("cons").find_elements_by_class_name("ugc")
            cons_data=[x.text for x in cons_list]

            temp_data=[company_title,title,name,company,score,pros_data,cons_data]
            pros_cons_data.append(temp_data)
        
        except:
            pass
        
        
    pros_cons_data_frame=pd.DataFrame(pros_cons_data)
    pros_cons_data_frame.columns=["Company Name","Review Title","Reviewer's Name","Reviewer's Company","Score","Pros","Cons"]
    
    #--------------------------------------------------------------------------------------
    
    
    #likelyhood to recommend
    browser.get('{}products/{}/reviews/likelihood-to-recommend'.format(source_site_title, company_title))
    likely_recommend_data=[]

    data_list= browser.find_elements_by_class_name("serp-details")

    for review in data_list:
        try:

            name=review.find_element_by_class_name("name").text
            recommend_data=review.find_element_by_class_name("question-response-container").text
            likely_recommend_data.append([name,recommend_data])

        except:
            pass

    likely_recommend_data_frame=pd.DataFrame(likely_recommend_data)
    likely_recommend_data_frame.columns=["Reviewer's Name","Likelihood To Recommend"]


    #------------------------------------------------------------------------------------------------------------
    #use case deployment
    browser.get('{}products/{}/reviews/product-usage'.format(source_site_title, company_title))
    data_list= browser.find_elements_by_class_name("serp-details")
    product_usage_data=[]
    for review in data_list:
        try:
            name=review.find_element_by_class_name("name").text
            scope_data=review.find_element_by_class_name("ugc").text
            product_usage_data.append([name,scope_data])
        except:
            pass
    product_usage_data_frame=pd.DataFrame(product_usage_data)
    product_usage_data_frame.columns=["Reviewer's Name","Use Cases and Deployment Scope"]
    #-----------------------------------------------------------------------------------


    # alternative considered
    browser.get('{}products/{}/reviews/alternatives-evaluated'.format(source_site_title, company_title))
    data_list= browser.find_elements_by_class_name("serp-details")
    alternative_eval_data=[]
    for review in data_list:
        try:
            name=review.find_element_by_class_name("name").text
            alternative_data=review.find_element_by_class_name("question-response-container").text
            alternative_eval_data.append([name,alternative_data])
        except:
            pass
    alternative_eval_data_frame=pd.DataFrame(alternative_eval_data)
    alternative_eval_data_frame.columns=["Reviewer's Name","Alterantives Considered"]

    #-------------------------------------------------------------------------------------------------

    #other software used
    browser.get('{}products/{}/reviews/other-software-used'.format(source_site_title, company_title))
    data_list= browser.find_elements_by_class_name("serp-details")
    other_software_data=[]
    for review in data_list:
        try:
            name=review.find_element_by_class_name("name").text
            other_software_used=review.find_element_by_class_name("question-response-container").text
            other_software_data.append([name,other_software_used])
        except:
            pass
    other_software_data_frame=pd.DataFrame(other_software_data)
    other_software_data_frame.columns=["Reviewer's Name","Other Software Used"]

    #merge all
    data_main=pd.merge(pros_cons_data_frame,likely_recommend_data_frame,on="Reviewer's Name",how="left")
    data_main=pd.merge(data_main,product_usage_data_frame,on="Reviewer's Name",how="left")
    data_main=pd.merge(data_main,alternative_eval_data_frame,on="Reviewer's Name",how="left")
    data_main=pd.merge(data_main,other_software_data_frame,on="Reviewer's Name",how="left")
    
    
    
    #final
    data_final=pd.concat([data_final,data_main]).reset_index(drop=True)
browser.close()
data_final.to_csv("companies_trusted_scrapped_data.csv",index=False)